In [1]:
from cassandra.cluster import Cluster
from ssl import SSLContext, PROTOCOL_TLSv1_2, CERT_REQUIRED
from cassandra_sigv4.auth import SigV4AuthProvider
import boto3

# ssl setup
ssl_context = SSLContext(PROTOCOL_TLSv1_2)
ssl_context.load_verify_locations('sf-class2-root.crt')  # change your file path for locating the certificate
ssl_context.verify_mode = CERT_REQUIRED

# boto3 session setup
boto_session = boto3.Session(region_name="us-east-2")  # this AWS credentials is specific to `us-east-2` region

C:\Users\rpbea\AppData\Local\Temp\ipykernel_7276\332746716.py:7: DeprecationWarning: ssl.PROTOCOL_TLSv1_2 is deprecated
  ssl_context = SSLContext(PROTOCOL_TLSv1_2)


In [2]:
# authorization setup with SigV4
auth_provider = SigV4AuthProvider(boto_session)

In [3]:
#cluster setup 
cluster = Cluster(['cassandra.us-east-2.amazonaws.com'], 
                  ssl_context=ssl_context, 
                  auth_provider=auth_provider, 
                  port=9142)  # TLS only communicates on port 9142

In [4]:
# establishing connection to Keyspace
session = cluster.connect()

In [7]:
# Insert any CQL queries between .connect() and .shutdown()

# For example, show all keyspaces created
r = session.execute('''
    SELECT * FROM system_schema.keyspaces;
    ''')
from pprint import pprint
pprint(r.current_rows)

[Row(keyspace_name='system_schema', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])),
 Row(keyspace_name='system_schema_mcs', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])),
 Row(keyspace_name='system', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])),
 Row(keyspace_name='system_multiregion_info', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])),
 Row(keyspace_name='de300_axc4282', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])),
 Row(keyspace_name='de300_demo', durable_writes=True, replication=Ordered

In [16]:
# For example, create a keyspace for HW2
r = session.execute('''
    CREATE KEYSPACE IF NOT EXISTS ryan_beam 
    WITH replication = {'class': 'SingleRegionStrategy'};
    ''')
print(r.current_rows)

[]


In [17]:
from cassandra.cluster import ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra import ConsistencyLevel


# Define execution profile with LOCAL_QUORUM
execution_profile = ExecutionProfile(
    consistency_level=ConsistencyLevel.LOCAL_QUORUM
)

# Cluster setup with correct profile
cluster = Cluster(
    ['cassandra.us-east-2.amazonaws.com'],
    ssl_context=ssl_context,
    auth_provider=auth_provider,
    port=9142,
    execution_profiles={EXEC_PROFILE_DEFAULT: execution_profile}
)

# establishing connection to Keyspace
session = cluster.connect()
session.set_keyspace('ryan_beam')  # Replace with your keyspace

In [19]:
# show all tables
r = session.execute("""
    SELECT * FROM system_schema.tables WHERE keyspace_name = 'ryan_beam';
""")
pprint(r.current_rows)

[]


In [20]:
session.execute("""
CREATE TABLE IF NOT EXISTS github (
    id UUID PRIMARY KEY,
    name TEXT,
    username TEXT
)
""")

In [21]:
import uuid

session.execute("""
    INSERT INTO github (id, name, username)
    VALUES (%s, %s, %s)
""", (uuid.uuid4(), "Ryan Beam", "rpbeam04"))

# Replace "Your_Name" with your name
# Replace "Your_GitHub_User_Name" with your real GitHub User

In [22]:
import csv

rows = session.execute("SELECT * FROM github")
with open("github.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["id", "name", "username"])
    for row in rows:
        writer.writerow([row.id, row.name, row.username])

In [23]:
# writing my credentials to main demo table
session.set_keyspace('de300_demo') 
session.execute("""
    INSERT INTO github (id, name, username)
    VALUES (%s, %s, %s)
""", (uuid.uuid4(), "Ryan Beam", "rpbeam04"))

r = session.execute("""
    SELECT * FROM Github
""")
pprint(r.current_rows)

[Row(id=UUID('909984c0-d6b7-402f-a0e9-1c002b9d9473'), name='Isabe_Zhang', username='ijzhang2021'),
 Row(id=UUID('84c930a8-99b8-4dd5-8e39-51b6fcbabed0'), name='Aaryan Agrawal', username='2409aaryan'),
 Row(id=UUID('983aa805-8739-45e5-81fc-605ee1b86953'), name='Katie Mowry', username='katiemowry'),
 Row(id=UUID('fdec6238-dbaf-4fd3-ad8e-8b9e73f4a069'), name='Your_Name', username='Your_GitHub_Username'),
 Row(id=UUID('9a76a9c1-d3f5-4b15-b5a2-11076dd99a35'), name='Alicia_Malek', username='am604'),
 Row(id=UUID('1a3c69aa-4d94-457a-9ea4-e2170154e3cc'), name='Nayeon Kim', username='nkimm26'),
 Row(id=UUID('e1ee1618-4a3c-4d98-a456-53a467db821f'), name='Harrison_Gillespie', username='Warry13'),
 Row(id=UUID('f379fad9-88aa-4313-9e58-84d0e95965fb'), name='Linda Liu', username='HYTF'),
 Row(id=UUID('596771cf-b1b3-43ae-96e8-b230d6d71139'), name='Aaryan Agrawal', username='2409aaryan'),
 Row(id=UUID('4242cf59-0e18-4569-98a2-79e5ed24f4a5'), name='Jonathan Wu', username='jwu5711'),
 Row(id=UUID('3d5075

In [27]:
r = session.execute("""
    SELECT * FROM Github;
""")
pprint(r.current_rows[20:30])

[Row(id=UUID('41e56080-9026-443a-99a3-e0e00f28ee4f'), name='Samnang Nuonsinoeun', username='SamFromCam'),
 Row(id=UUID('790b7baf-44a4-487e-8dbf-42f528a6edcd'), name='Alex Chen', username='chenisius123'),
 Row(id=UUID('fa13a2af-bf53-4ebe-848e-27aa86092d9e'), name='Ryan Beam', username='rpbeam04'),
 Row(id=UUID('a2772956-cc70-4462-a6d6-c3e409696e99'), name='Seeley McGillis', username='seeleymcgillis'),
 Row(id=UUID('72afbf39-ef82-4e76-9fe3-7ff7f0bd0da6'), name='Tidapa Thampeera', username='nakatidapa'),
 Row(id=UUID('57e34312-6136-4a58-8b8a-6c4ae8670269'), name='Isabel_Zhang', username='ijzhang2021'),
 Row(id=UUID('62355580-aae7-405a-a566-4c833a1e8681'), name='Sabian Atmadja', username='sabian147'),
 Row(id=UUID('29b45fd4-2864-42af-89ba-21d5639bf6f1'), name='Isabel_Zhang', username='ijzhang2021'),
 Row(id=UUID('107ef784-861c-45a9-a310-38d5103d5f96'), name='Sabian Atmadja', username='sabian147'),
 Row(id=UUID('f0df0032-8b4a-47f8-ac2c-41ff61dca42b'), name='Kyan Shlipak', username='kyanshli